# FIT5196 Assessment 1
#### Student Name: Aishwarya Srirangam Murali 
#### Student ID: 29999715

Date: 18/11/2020


Libraries used:
* pandas 0.19.2 (for data frame, included in Anaconda Python 3.6) 
* re 2.2.1 (for regular expression, included in Anaconda Python 3.6) 
* tabula to read tables in PDF)
* geopandas (Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and descartes and matplotlib for plotting)
* math (for mathematical functions)
* numpy (adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays) 
* potly (to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts)

In [ ]:
!pip install tabula-py --user
!pip install gtfs_kit --user
!pip install geopandas --user
!pip install plotly --user
!pip install pandas --user
!pip install numpy --user
!pip install Shapely --user

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
%matplotlib inline

In [ ]:
import re
import tabula

import pandas as pd
import numpy as np
import geopandas as gpd

from shapely.geometry import Point, Polygon
from math import radians, cos, sin, asin, sqrt 
from collections import Counter 

from pathlib import Path
from matplotlib import pyplot as plt


def ls(path): return list(path.iterdir())

import zipfile

def extract_file(path_to_zip, extract_to):  
    with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

In [ ]:
if Path('29999715.zip').exists():
    extract_file('29999715.zip', '29999715')
path = Path('./29999715/')
ls(path)

# Data Integration

## Victoria Suburbs Boundary data
* Reading the vic_suburb_boundary shape file 
* Checking for duplicates to check if the suburbs are repeated 
* Utilities to plot the points on victoria suburbs map

In [ ]:
if Path('vic_suburb_boundary.zip').exists(): 
    extract_file('vic_suburb_boundary.zip', 'vic_suburb_boundary')
ls(Path('vic_suburb_boundary'))

In [ ]:
vic_sub = gpd.read_file('vic_suburb_boundary/VIC_LOCALITY_POLYGON_shp.shp')
vic_sub.head()

In [ ]:
vic_sub.info()

In [ ]:
#check for duplicates
for c in vic_sub.columns:
    print(c, (vic_sub.drop_duplicates([c]).shape))

No duplicates in the geometry column, that means, the suburbs are not repeated

In [ ]:
#Utilities to plot on map

def add_pos_col(df):
    df.loc[:, "geometry"] = df[["lng", "lat"]].apply(lambda p: Point(p[0], p[1]), axis=1)
    
def get_geodf(df, epsg1=4283, epsg2=None):
    add_pos_col(df)
    return gpd.GeoDataFrame(df).set_geometry('geometry')

def plot_on_map(base_map, pts=None, ax=None, figsize=(10, 10), map_options={"color":"gray"}, 
                points_options = {"column":None, "legend":True, "color":"blue"}, filename=None):
    
    if ax==None: fig, ax = plt.subplots(1, 1, figsize=figsize)
    base_map = base_map.copy()
    base_map["index"] = base_map.index
    base_map.plot(ax=ax, **map_options)
    if pts is not None: 
        pts = pts.copy()
        get_geodf(pts).plot(ax=ax, **points_options)
    if filename: 
        print(f"Saving {filename}")
        plt.savefig(filename, figure=fig, bbox_inches='tight', pad_inches = 0.2)
    return ax          

In [ ]:
plot_on_map(vic_sub)
plt.title('Victoria Suburbs')

## Hospital Data
* Reading the hospitals.pdf file 
* Checking for duplicates 
* Checking if the ids are right 

In [ ]:
hospitals = tabula.read_pdf(path/"hospitals.pdf", pages='all')
hospitals = pd.concat([df[['id', 'lat', 'lng', 'name']] for df in hospitals[:5]]) #concat all data from the 5 pages of the pdf file
hospitals = hospitals.dropna(how='all').reset_index(drop=True) #remove the empty rows
hospitals.info()

In [ ]:
#check for duplicates
for c in hospitals.columns:
    print(c, hospitals.shape, (hospitals.drop_duplicates([c]).shape))

In [ ]:
c = Counter(np.array(hospitals['name'].str.lower()))
c.most_common(10)

Checked with the pdf file, indeed there are 4 records with the name 'Waverley Private Hospital' and 2 records with the name 'Epsworth Eastern Hospital'

In [ ]:
hospitals[hospitals['name'] == 'Waverley Private Hospital']

In [ ]:
hospitals[hospitals['name'] == 'Epsworth Eastern Hospital']

On google search, I found that only one of the Epsworth Eastern hospital is real, same with Waverley private hospital; However, since we are not allowed to use any external data, its impossible to find out which ones we are supposed to keep, so I'll keep them all. Important thing to note here is that this might end up affecting the nearest hospital and distance to nearest hospital columns in the final schema.

In [ ]:
#Hospitals that do not exist
problematic_hospitals = ["hospital_140", "hospital_053", "hospital_057", "hospital_170"]

In [ ]:
id_pattern = re.compile("^hospital_[0-9]{3}$")
hospitals[hospitals['id'].apply(lambda i: not bool(id_pattern.match(i)))]

No problems with id

In [ ]:
hospitals[['lat', 'lng']].hist()

Seems within range, no other problems with hospital data

In [ ]:
hospitals.head()

In [ ]:
plot_on_map(vic_sub, hospitals)

## Shopping centers
* Reading shopingcenters.xlsx
* Checking for duplicates 
* Checking for repititions or missing values 
* Plotting to see how the values are distributed

In [ ]:
shp_center = pd.read_excel(path/'shopingcenters.xlsx', index_col=0)
shp_center.head(10)

In [ ]:
shp_center.info()

In [ ]:
#check for duplicate, columnwise
for c in shp_center.columns:
    print(c, (shp_center.drop_duplicates([c]).shape))

No duplicates!

In [ ]:
id_pattern = re.compile("^SC_[0-9]{3}$")
shp_center[shp_center['sc_id'].apply(lambda i: not bool(id_pattern.match(i)))]

no problems with sc_id

In [ ]:
shp_center[['lat', 'lng']].describe()

In [ ]:
shp_center[['lat', 'lng']].hist()

In [ ]:
plot_on_map(vic_sub, shp_center)

Looks like some of these shopping centers are at the other end of the world, we can't possibly fix these but we can remove them. However, there was no direction in the 'FIT5196-S2-2020 assessment 3.pdf' about such errors, so I am keeping them. 

## Supermarkets
* Reading supermarkets.html
* Checking if there are any irregularities with the id 
* Checking for duplicates 
* Checking for repititions or missing values 

In [ ]:
with open(path/'supermarkets.html') as f:
    supermarkets = pd.read_html(f, index_col=0)[0]

In [ ]:
supermarkets.info()

In [ ]:
supermarkets.head()

In [ ]:
id_pattern = re.compile("^S_[0-9]{3}$")
supermarkets[supermarkets['id'].apply(lambda i: not bool(id_pattern.match(i)))]

No problems with the id

In [ ]:
for c in supermarkets.columns:
    print(c, (supermarkets.drop_duplicates([c]).shape))

In [ ]:
c = Counter(np.array(supermarkets['lat']))
c.most_common(10)

In [ ]:
supermarkets[supermarkets['lat']==-37.86685]

This one supermarket is repeated, let's remove the S_227

In [ ]:
supermarkets = supermarkets.drop_duplicates(['lat', 'lng', 'type'], keep='first').reset_index(drop=True)

In [ ]:
supermarkets[supermarkets['lat']==-37.86685]

In [ ]:
c = Counter(np.array(supermarkets['lng']))
c.most_common(10)

In [ ]:
supermarkets[supermarkets['lng']==144.75218]

These two records with identical lng (144.75218) are far apart, so no problems there. 

In [ ]:
plot_on_map(vic_sub, supermarkets)

In [ ]:
supermarkets[['lat', 'lng']].hist()

Again, some supermarkets have problematic lat, long, we can't possibly fix them, so I am keeping them as it is in the dataset. 

In [ ]:
supermarkets['type'].unique()

No problems here either

## Real Estate
* Loading the xml and the json files 
* Removing all the duplicates 
* Concatenating them based on their rows to obtain complete real state dataset 

In [ ]:
#Fix the real state.xml file

with open(path/'real_state.xml', 'r+') as f:
    data = f.read()
    
with open(path/'real_state-fixed.xml', 'w') as f:
    f.write(data.strip('b')[1:-1])

In [ ]:
from bs4 import BeautifulSoup

file = path/'real_state-fixed.xml'
with open(file) as fp:
    soup = BeautifulSoup(fp, 'xml')

In [ ]:
cols = ['property_id', 'lat', 'lng', 'addr_street', 'price', 'property_type', 'year', 'bedrooms', 
        'bathrooms', 'parking_space']

In [ ]:
for c in cols:
    print(c, len(soup.findAll(c)), len(soup.findAll(c)[0].findChildren()))

In [ ]:
all_tags = [t.name for t in soup.findAll('property_id')[0].findChildren()]
xml_data = {t: {} for t in all_tags}

for c in cols:
    rows = soup.findAll(c)[0].findChildren()
    for r in rows:
        assert r.name in all_tags
        xml_data[r.name][c] = r.text 

In [ ]:
xml_df = pd.DataFrame(xml_data).transpose()
xml_df.reset_index(drop=True, inplace=True)
xml_df = xml_df.drop_duplicates()
xml_df.shape

In [ ]:
xml_df['lat'] = xml_df['lat'].astype(float)
xml_df['lng'] = xml_df['lng'].astype(float) 
for c in ['property_id', 'price', 'year', 'bedrooms', 'bathrooms', 'parking_space']:
    xml_df[c] = xml_df[c].astype(int)

In [ ]:
import json
with open(path/'real_state.json') as f:
    json_data = json.load(f)

In [ ]:
json_df = pd.DataFrame(json_data)
json_df = json_df.drop_duplicates()

In [ ]:
json_df.shape

In [ ]:
common_ids = xml_df[xml_df['property_id'].isin(json_df['property_id'])]['property_id']

In [ ]:
len(common_ids)

In [ ]:
for i in common_ids:
    a = list(xml_df[xml_df['property_id'] == i].transpose().to_dict().items())[0][1]
    b = list(json_df[json_df['property_id'] == i].transpose().to_dict().items())[0][1]
    if a != b:
        print('-'*50)
        for k, v in a.items():
            if b[k] != v:
                print(k, round(v, 6), round(b[k], 6))

The records with common property id are exactly the same, so we just have to combine the two data frames, while only keeping the common ids from one of them, so in total there will be 1002 (json) + 1005 (xml) - 25 (common) = 1982 rows

In [ ]:
real_state = pd.concat([xml_df, json_df[~ json_df['property_id'].isin(common_ids)]]).reset_index(drop=True)
real_state.shape

In [ ]:
real_state.head()

In [ ]:
for c in real_state.columns:
    print(c, real_state.shape, (real_state.drop_duplicates([c]).shape))

In [ ]:
c = Counter(np.array(real_state['lat']))
c.most_common(10)

In [ ]:
real_state[real_state['lat'].isin([o[0] for o in c.most_common(5)])].sort_values(by='lat')

In [ ]:
c = Counter(np.array(real_state['lng']))
c.most_common(10)

In [ ]:
real_state[real_state['lng'].isin([o[0] for o in c.most_common(5)])].sort_values(by='lng')

In [ ]:
c = Counter(np.array(real_state['addr_street']))
c.most_common(10)

In [ ]:
real_state[real_state['addr_street'].isin([o[0] for o in c.most_common(5)])].sort_values(by='addr_street')

We have already removed all the duplciates, and looking at the repititions in the lat/lng/addr_street column tells us that there aren't any rows with different property id but exact same everything else

In [ ]:
len(real_state['addr_street'].unique())

In [ ]:
len(real_state['addr_street'].str.lower().unique())

So no same add_street rows with different Capitalizations

In [ ]:
real_state[['lat', 'lng']].hist()

In [ ]:
plot_on_map(vic_sub, real_state)

Looks like the dataset is just for Melbourne, and suburbs

In [ ]:
id_pattern = re.compile("^[0-9]+$")
real_state[real_state['property_id'].astype(str).apply(lambda i: not bool(id_pattern.match(i)))]

property_id okay too

## GTFS data

Lets look at the structure of the zip file

In [ ]:
extract_file('GTFS_Melbourne_Train_Information.zip', 'GTFS_Melbourne_Train_Information')
ls(ls(ls(Path('GTFS_Melbourne_Train_Information'))[0])[0])

so the files are in `GTFS_Melbourne_Train_Information/1. GTFS - Melbourne Train Information - From PTV (9 Oct 2015)/GTFS - Melbourne Train Information/`

In [ ]:
class Feed(object):
    def __init__(self, zip_file):
        extract_file(zip_file, str(zip_file)[:-4])
        self.files_path = ls(ls(Path('GTFS_Melbourne_Train_Information'))[0])[0]
        print('All files: ', [f.name for f in ls(self.files_path)])
        
        self.agency = self.get_file('agency.txt')
        self.calendar = self.get_file('calendar.txt')
        self.calendar_dates = self.get_file('calendar_dates.txt')
        self.routes = self.get_file('routes.txt')
        self.shapes = self.get_file('shapes.txt')
        self.stops = self.get_file('stops.txt')
        self.stop_times = self.get_file('stop_times.txt')
        self.trips = self.get_file('trips.txt')
        
    def get_file(self, filename):
        return pd.read_csv(self.files_path/filename)

In [ ]:
feed = Feed('GTFS_Melbourne_Train_Information.zip')

In [ ]:
feed.stops[['stop_lat', 'stop_lon']].hist()

In [ ]:
for c in feed.stops.columns:
    print(c, (feed.stops.drop_duplicates([c]).shape))

## combine data

In [ ]:
df = real_state.copy()
df.head()

### Shortest ids and distances

In [ ]:
#Haversine
def distance_in_kms(lat1, long1, lat2, long2): 
    lat1, long1, lat2, long2 = radians(lat1), radians(long1), radians(lat2), radians(long2)    
    d = sin((lat2 - lat1) / 2)**2 + cos(lat1) * cos(lat2) * sin((long2-long1) / 2)**2
    d = 2 * asin(sqrt(d)) * 6378
    return d


def min_distance(r, dist_from):
    plat, plng = r['lat'], r['lng']
    dist_from['dist'] = dist_from.apply(lambda x: distance_in_kms(float(x['lat']), float(x['lng']), plat, plng), axis=1)
    sorted_dist_from = dist_from.sort_values(by='dist').reset_index(drop=True)
    if sorted_dist_from.loc[0]['dist'] == sorted_dist_from.loc[1]['dist']:
        print('-'*100)
        print(r)
        print('-'*50)
        print(sorted_dist_from.loc[0])
        print('-'*50)
        print(sorted_dist_from.loc[1])
    assert (sorted_dist_from.loc[0]['lat'] < sorted_dist_from.loc[0]['lng'])
    assert (sorted_dist_from.loc[0]['lat'] < 30)
    assert (sorted_dist_from.loc[0]['lng'] > 140)
    return sorted_dist_from.loc[0]

Add the Shopping_center_id for the nearest shopping center with Distance_to_sc

In [ ]:
df['Shopping_center_id'] = df.apply(lambda r: min_distance(r, shp_center.copy())['sc_id'], axis=1)
df['Distance_to_sc'] = df.apply(lambda r: round(min_distance(r, shp_center.copy())['dist'], 3), axis=1)

In [ ]:
df.head()

Add the Hospital_id for the nearest hospital with Distance_to_hospital

In [ ]:
df['Hospital_id'] = df.apply(lambda r: min_distance(r, hospitals.copy())['id'], axis=1)
df['Distance_to_hospital'] = df.apply(lambda r: round(min_distance(r, hospitals.copy())['dist'], 3), axis=1)

In [ ]:
df.head()

In [ ]:
df[df['Hospital_id'].isin(problematic_hospitals)].shape

This is problematic, but since we can't change anything in the hospitals file, we will just let it be

Now, Add the Supermarket_id for the nearest supermarket with Distance_to_supermaket

In [ ]:
df['Supermarket_id'] = df.apply(lambda r: min_distance(r, supermarkets.copy())['id'], axis=1)
df['Distance_to_supermaket'] = df.apply(lambda r: round(min_distance(r, supermarkets.copy())['dist'], 3), axis=1)

In [ ]:
df.head()

Add the Train_station_id for the nearest train station with Distance_to_train_station

In [ ]:
train_stops = feed.stops.copy()
train_stops['lat'] = train_stops['stop_lat']
train_stops['lng'] = train_stops['stop_lon']
train_stops.head()

In [ ]:
df['Train_station_id'] = df.apply(lambda r: min_distance(r, train_stops.copy())['stop_id'], axis=1)
df['Distance_to_train_station'] = df.apply(lambda r: round(min_distance(r, train_stops.copy())['dist'], 3), axis=1)

In [ ]:
df.head()

### Suburb

Let's find and all suburbs now, the `VIC_LOCA_2` column represents the suburb name in the vic_suburb data

In [ ]:
def find_suburb(r, suburbs):
    plat, plng = r['lat'], r['lng']
    p = Point(float(plng), float(plat))
    suburb = []
    suburbs['belong_here'] = suburbs.apply(lambda g: (g.geometry.contains(p) or g.geometry.touches(p)), axis=1)
    suburb = suburbs[suburbs['belong_here'] == True]
    assert len(suburb) <= 1
    if len(suburb) == 1:
        return suburb.iloc[0]
    return "not available"

In [ ]:
# A few tests, compared with google maps
%time assert find_suburb(df.iloc[0], vic_sub.copy())['VIC_LOCA_2'] == "MOOROOLBARK"
%time assert find_suburb(df.iloc[5], vic_sub.copy())['VIC_LOCA_2'] == "MENTONE"
%time assert find_suburb(df.iloc[199], vic_sub.copy())['VIC_LOCA_2'] == "LALOR"
%time assert find_suburb(df.iloc[1599], vic_sub.copy())['VIC_LOCA_2'] == "SEDDON"

In [ ]:
df['suburb'] = df.apply(lambda r: find_suburb(r, vic_sub.copy())["VIC_LOCA_2"], axis=1)

In [ ]:
df.head()

In [ ]:
df[df['suburb'] == 'not available'].shape

All suburbs are available

### travel_min_to_CBD and Transfer_flag columns

In [ ]:
feed.stops[feed.stops['stop_name'].str.contains('Flinders Street')]

In [ ]:
possible_trips = list(feed.trips[feed.trips['service_id'] == 'T0']['trip_id'])

We only want trips with service id T0 because it runs on all days

In [ ]:
def get_in_min(t):
    h, m, s = list(map(int, t.split(':')))
    return (h*60 + m + s/60)

In [ ]:
stop_times = feed.stop_times
stop_times['arrival_min'] = stop_times['arrival_time'].apply(get_in_min)
stop_times['departure_min'] = stop_times['departure_time'].apply(get_in_min)
stop_times['stop_id'] = stop_times['stop_id'].astype(str)

In [ ]:
from tqdm import tqdm_notebook

### In the below code, 

* Stop times to filter the trips with stops which is nearest to the property and the stop id of flinder's street 
* Filtering out the trips that depart between 7am to 9pm from the stop_id and whose arrival times at the city centre are after the departure times from the stop nearest to the property

In [ ]:
def trips_stop2city(stop_id):
    trips_from_stop = stop_times[(stop_times['stop_id'] == str(stop_id)) & (stop_times['departure_min'] >=420) & (stop_times['departure_min'] <= 540) & (stop_times['trip_id'].isin(possible_trips))].reset_index(drop=True)    
    trips_from_city = stop_times[(stop_times['stop_id'] == "19854")]
    common_trips = list(trips_from_city[trips_from_city['trip_id'].isin(list(trips_from_stop['trip_id']))]['trip_id'])
    
    trips_from_city.index = trips_from_city['trip_id']
    trips_from_stop.index = trips_from_stop['trip_id']
    
    
    #Filter common trips in same sequence
    trips_from_city = trips_from_city.loc[common_trips].reset_index(drop=True)
    trips_from_stop = trips_from_stop.loc[common_trips].reset_index(drop=True)
    
    assert list(trips_from_stop['trip_id']) == list(trips_from_city['trip_id'])
    
    fil = (trips_from_stop['departure_min'] <= trips_from_city['departure_min']) & (trips_from_stop['stop_sequence'].astype(int) < trips_from_city['stop_sequence'].astype(int))
    trips_from_stop_to_city = trips_from_stop[fil].reset_index(drop=True)
    trips_to_city_from_stop = trips_from_city[fil].reset_index(drop=True)
    return trips_from_stop_to_city, trips_to_city_from_stop

In [ ]:
for stop_id in tqdm_notebook(df['Train_station_id']):
    trips_from_stop_to_city, _ = trips_stop2city(stop_id)
    if len(trips_from_stop_to_city) == 0:
        print(stop_id)

This means that there is at least one direct trip (departing between 7am to 9am)that with service id T0 between all stations and the Flinder's Street except for the stops 20027 and 19854 (this one is flinder's street itself), we can ignore the station 20027 as it appears only 4 times in the total dataset, so we have only considered the direct trips

In [ ]:
stop_to_city, city_from_stop  = trips_stop2city('19948')

In [ ]:
def get_travel_time(r, stop_times):
    stop_id = r['Train_station_id']
    if str(stop_id) == '19854':
        return 0, 0 #Nearest Stop is city, so no tran
    else:
        stop_to_city, city_from_stop  = trips_stop2city(stop_id)
        if len(stop_to_city) == 0:
            return 0, -1 #Default values 
        else:
            travel_times = city_from_stop['arrival_min'] - stop_to_city['departure_min']
            assert (city_from_stop['departure_min'] < 24*60).sum() == city_from_stop.shape[0]
            return np.floor(travel_times.mean()), 0 #all direct trips only

In [ ]:
df['travel_min_to_CBD'] = df.apply(lambda r: get_travel_time(r, stop_times)[0], axis=1)
df['Transfer_flag'] = df.apply(lambda r: get_travel_time(r, stop_times)[1], axis=1)

In [ ]:
df.head()

In [ ]:
df[df['Transfer_flag'] == -1]

Only four rows were left with default values

## Save the combined data

In [ ]:
req_columns = ['property_id', 'lat', 'lng', 'addr_street',
                'suburb', 'property_type', 'year', 'bedrooms',
                'bathrooms', 'parking_space', 'Shopping_center_id', 
                'Distance_to_sc', 'Train_station_id', 'Distance_to_train_station',
                'travel_min_to_CBD', 'Transfer_flag', 'Hospital_id',
                'Distance_to_hospital', 'Supermarket_id', 'Distance_to_supermaket']

In [ ]:
df[req_columns].to_csv('29999715_A3_solution.csv', index=False)

In [ ]:
submission = pd.read_csv('29999715_A3_solution.csv')
submission.head()

In [ ]:
for c in submission.columns:
    assert c in req_columns
    
for r in req_columns:
    assert r in submission.columns

# Reshape Data

In [ ]:
import math
import plotly.express as px
from sklearn import preprocessing

In [ ]:
predictor_variables = ['Distance_to_sc', 'travel_min_to_CBD', 'Distance_to_hospital']
target_variable = 'price'
variables_of_interes = predictor_variables + [target_variable]

In [ ]:
df[predictor_variables].describe()

These predictor variables have been measured at different scales. i.e., `Distance_to_sc` in km, `travel_min_to_CBD` in minutes, and `Distance to hsopital` in km, therefore, we need to scale them first to make comparisons, it is generally not required to transform the output variable.

### Std and MinMax scaling

In [ ]:
std_scaler = preprocessing.StandardScaler().fit(df[predictor_variables])
std_values = std_scaler.transform(df[predictor_variables])
std_values, std_values.shape 

In [ ]:
std_predictor_variables = [f'std_{v}' for v in predictor_variables]
for i, v in enumerate(std_predictor_variables):
    df[v] = std_values[:, i]

We plan to check the normality of the distributions of the variable and hence we many need to transform the variables. It will help in transformation if our variables have values that are strictly positive. Therefore, let's use the `feature_range` option in the Min Max scaler to ensure that we have strictly positive values. 

In [ ]:
minmax_scaler = preprocessing.MinMaxScaler(feature_range=(1, 2)).fit(df[predictor_variables])
minmax_values = minmax_scaler.transform(df[predictor_variables])

minmax_predictor_variables = [f'minmax_{v}' for v in predictor_variables]
for i, v in enumerate(minmax_predictor_variables):
    df[v] = minmax_values[:, i]

In [ ]:
df[predictor_variables + std_predictor_variables + minmax_predictor_variables].describe()

Since we want to apply boxcox transformation, we need to take the minmax normalized data as its values start from zero and boxcox transformation requires values>0

In [ ]:
pref = 'minmax'
independant_vars_dict = {'': predictor_variables, 'std': std_predictor_variables,
                        'minmax': minmax_predictor_variables}

independent_vars = independant_vars_dict[pref]

## Normality of the continous variables

Algorithms like linear regression and logistic regression explicitly assume the real-valued variables have a Gaussian distribution and therefore, its important that we check if our input and output variables have gaussian distribution. Let's also compare with the distributions of the box-cox and square root (sqrt) transformed values of the respective variables. We have also looked at skew values and their changes as we use box-cox and sqrt transformation. 


In [ ]:
from scipy import stats 
import seaborn as sns 
import matplotlib.pyplot as plt 

In [ ]:

def transform_boxcox(og_data, name):
    tfmd_data, tfm_lambda = stats.boxcox(og_data)
    fig, ax = plt.subplots(1, 2)
    kwargs_for_plotting = dict(hist=True, kde=True, 
                           kde_kws = {'shade': True, 'linewidth': 2},
                           color='blue')
    
    sns.distplot(og_data, label='Original data', ax=ax[0], **kwargs_for_plotting)
    sns.distplot(tfmd_data, label='BoxCox Transformed data', ax=ax[1], **kwargs_for_plotting)
    
    ax[0].set_xlabel(name); ax[1].set_xlabel(name);
    plt.legend(loc = "upper right") 
    fig.set_figheight(5) 
    fig.set_figwidth(10) 
    
    print('Lambda value: ', tfm_lambda)
    print('Change in skew', stats.skew(og_data), "-->", stats.skew(tfmd_data))
    return tfmd_data, tfm_lambda


def transform_sqrt(og_data, name):
    tfmd_data = np.sqrt(og_data)
    fig, ax = plt.subplots(1, 2)
    kwargs_for_plotting = dict(hist=True, kde=True, 
                           kde_kws = {'shade': True, 'linewidth': 2},
                           color='blue')
    
    sns.distplot(og_data, label='Original data', ax=ax[0], **kwargs_for_plotting)
    sns.distplot(tfmd_data, label='Sqrt Transformed data', ax=ax[1], **kwargs_for_plotting)
    
    ax[0].set_xlabel(name); ax[1].set_xlabel(name);
    plt.legend(loc = "upper right") 
    fig.set_figheight(5) 
    fig.set_figwidth(10) 
    
    print('Change in skew', stats.skew(og_data), "-->", stats.skew(tfmd_data))
    return tfmd_data, None


def transform_log(og_data, name):
    tfmd_data = np.log(og_data)
    fig, ax = plt.subplots(1, 2)
    kwargs_for_plotting = dict(hist=True, kde=True, 
                           kde_kws = {'shade': True, 'linewidth': 2},
                           color='blue')
    
    sns.distplot(og_data, label='Original data', ax=ax[0], **kwargs_for_plotting)
    sns.distplot(tfmd_data, label='Log Transformed data', ax=ax[1], **kwargs_for_plotting)
    
    ax[0].set_xlabel(name); ax[1].set_xlabel(name);
    plt.legend(loc = "upper right") 
    fig.set_figheight(5) 
    fig.set_figwidth(10) 
    
    print('Change in skew', stats.skew(og_data), "-->", stats.skew(tfmd_data))
    return tfmd_data, None

In [ ]:
df["bc_" + independent_vars[0]] = transform_boxcox((df[independent_vars[0]]), independent_vars[0])[0]
df["bc_" + independent_vars[1]] = transform_boxcox((df[independent_vars[1]]), independent_vars[1])[0]
df["bc_" + independent_vars[2]] = transform_boxcox((df[independent_vars[2]]), independent_vars[2])[0]
df['bc_price'] = transform_boxcox(df['price'], 'price')[0]

So the boxcox transformation helps a lot in transforming the distributions to normal for the variables of with Distance_to_sc, Distance_to_hospital, travel_min_to_CBD and price. There are significant changes in the values of skew and the distribution of the transformed values look much more normal compared to that of the non-transfomred values. Let's also try the sqrt transformation.

In [ ]:
df["sqrt_" + independent_vars[0]] = transform_sqrt(df[independent_vars[0]], independent_vars[0])[0]
df["sqrt_" + independent_vars[1]] = transform_sqrt(df[independent_vars[1]], independent_vars[1])[0]
df["sqrt_" + independent_vars[2]] = transform_sqrt(df[independent_vars[2]], independent_vars[2])[0]
df['sqrt_price'] = transform_sqrt(df['price'], 'price')[0]

So the square root transformation helps in transforming the distributions of with Distance_to_sc, Distance_to_hospital. But it is not as good while transforming the distribution of the output variable Price and the input variable tavel_min_to_CBD. Let us also consider the log transformation. 

In [ ]:
df["log_" + independent_vars[0]] = transform_log(df[independent_vars[0]], independent_vars[0])[0]
df["log_" + independent_vars[1]] = transform_log(df[independent_vars[1]], independent_vars[1])[0]
df["log_" + independent_vars[2]] = transform_log(df[independent_vars[2]], independent_vars[2])[0]
df['log_price'] = transform_log(df['price'], 'price')[0]

Neither log nor sqrt transformations are as good as the box-cox transformation. This is expected as the box-cox transformation is more adaptive. So let us use the box-cox transformation only.

## Linearity

A linear model will try to fit a straight line through the data points given. Therefore, it is important to ensure that there is a linear relationship between the independent variable and the dependent variables. We plot each of the independ variable against he dependent variable to see if there is a linear relationship. We also plot a trendline using ols regression and look the r-squared value of that. 

In [ ]:
r2_values = []

In [ ]:
fig = px.scatter(df, x="bc_"+independent_vars[0], y="bc_price", trendline="ols")
fig.show()

results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
r2_values.append(float(pd.DataFrame(results.tables[0]).iloc[0, 3].data))

In [ ]:
fig = px.scatter(df, x="bc_"+independent_vars[1], y="bc_price", trendline="ols")
fig.show()


results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
r2_values.append(float(pd.DataFrame(results.tables[0]).iloc[0, 3].data))

In [ ]:
fig = px.scatter(df, x="bc_"+independent_vars[2], y="bc_price", trendline="ols")
fig.show()


results = px.get_trendline_results(fig)
results = results.iloc[0]["px_fit_results"].summary()
r2_values.append(float(pd.DataFrame(results.tables[0]).iloc[0, 3].data))

In [ ]:
r2_values

There is a linear relationship between the predictor variables and the target variable. This linear relationship is more pronounced in case of the input variables Distance_to_hospital and travel_min_to_CBD (Both min max normalized and box cox transformed). Therfore the condition of Linearity is satisfied.

## Normality of the Residuals

### Build a linear Model

Now, lets build a model and look at the residuals to ensure the condition of normality is maintained, we will use the box cox transformed values of min max normalized input variables with an offset value of 1, and the box cox transformed output variable price

In [ ]:
def get_results(model, X, y):
    preds = model.predict(X)
    results = pd.DataFrame({'Actual': y, 'Predicted': preds})
    results['Residuals'] = np.abs(results['Actual']) - np.abs(results['Predicted'])
    return results

In [ ]:
from sklearn.linear_model import LinearRegression

Let's try one model with transfomred output variable and one model with non-transformed output variable

In [ ]:
#Model with transformed output variable

X, y = df[["bc_"+iv for iv in independent_vars]], df['bc_price']

model = LinearRegression()
model.fit(X, y)

r2 = model.score(X, y)
print(r2)

In [ ]:
#Model with non transformed output variable

X, y_non_tfmd = df[["bc_"+iv for iv in independent_vars]], df['price']

model_non_tfmd = LinearRegression()
model_non_tfmd.fit(X, y_non_tfmd)

r2 = model_non_tfmd.score(X, y_non_tfmd)
print(r2)

### Check normality

In [ ]:
def plot_hist(data):
    kwargs_for_plotting = dict(hist=True, kde=True, 
                           kde_kws = {'shade': True, 'linewidth': 2},
                           color='blue')
    
    sns.distplot(data, **kwargs_for_plotting)

In [ ]:
results = get_results(model, X, y)
# results['Residuals'].plot.hist()
plot_hist(results['Residuals'])
plt.xlabel('Residuals')
plt.title('Model with both input and output variables transformed')

In [ ]:
results = get_results(model_non_tfmd, X, y_non_tfmd)
plot_hist(results['Residuals'])
plt.xlabel('Residuals')
plt.title('Model with only input variables transformed')

From the above two graphs it is clear that the model with both input and outputs transformed satisfies the normality assumption better

### References 
* Tutorials — pandas 0.15.2 documentation. (2020). Retrieved 18 November 2020, from https://pandas.pydata.org/pandas-docs/version/0.15/tutorials.html
* tabula-py. (2020). Retrieved 18 November 2020, from https://pypi.org/project/tabula-py/
* GeoPandas 0.8.0 — GeoPandas 0.8.0 documentation. (2020). Retrieved 18 November 2020, from https://geopandas.org/
* math — Mathematical functions — Python 3.9.0 documentation. (2020). Retrieved 18 November 2020, from https://docs.python.org/3/library/math.html
* Overview — NumPy v1.19 Manual. (2020). Retrieved 18 November 2020, from https://numpy.org/doc/stable/ 
* Plotly Python Graphing Library. (2020). Retrieved 18 November 2020, from https://plotly.com/python/